In [6]:
def readfile(file):
    with open(file) as f:
        contents = f.read()[:-1].split('\n\n')
    
    mapping_dictionary = {}

    seed_data = contents[0].split(": ")
    map_data = contents[1:]
    mapping_dictionary[seed_data[0]] = [int(i) for i in seed_data[1].split()]
    
    map_content_dictionary = {}
    for i in map_data:
        map_content_data = i.split(" map:\n")
        map_content_list = []
        for map in map_content_data[1].split("\n"):
            mapping_rule = {0:'destination_range_start',1:'source_range_start',2:'range_length'}
            mapping_content = {}
            for j in mapping_rule.keys():
                mapping_content[mapping_rule[j]] = int(map.split()[j])
            map_content_list.append(mapping_content)
        map_content_dictionary[map_content_data[0]] = map_content_list
    
    mapping_dictionary['mapping'] = map_content_dictionary

    return mapping_dictionary

def seed_mapper(identified_value, source, mapping):
    current_map_key = [i for i in mapping.keys() if i.split('-to-')[0]==source][0]
    print(current_map_key)
    active_map = mapping[current_map_key]

    new_identified_value = identified_value

    for map in active_map:
        if identified_value>=map['source_range_start'] and identified_value<map['source_range_start']+map['range_length']:
            new_identified_value = map['destination_range_start']+identified_value-map['source_range_start']
            break

    next_destination = current_map_key.split('-to-')[1]

    return new_identified_value if next_destination=='location' else seed_mapper(new_identified_value, next_destination, mapping)

x = readfile('input.txt')

y = seed_mapper(x['seeds'][0],'seed',x['mapping'])
y

seed-to-soil
soil-to-fertilizer
fertilizer-to-water
water-to-light
light-to-temperature
temperature-to-humidity
humidity-to-location


881943038